# Лабораторна робота 3 з "Асиметричних криптосистем та протоколів"
## Тема: Криптосистема Рабіна; Атака на протокол доведення знання без розголошення

**Виконали**\
Дигас Богдан, ФІ-03\
Починок Юрій, ФІ-03

## Підключаємо бібліотеки

In [24]:
import random
rand = random.SystemRandom()
from sympy.ntheory import jacobi_symbol, legendre_symbol
import math

## bit length = $256$

In [ ]:
global bit_length = 256

## Допоміжні функції

In [25]:

def decomposing_number(n, a):
    exp = n - 1
    while not exp & 1:  # while exp is even
        exp >>= 1  # divide by 2
    if pow(a, exp, n) == 1:
        return True  # number is composite
    while exp < n - 1:
        if pow(a, exp, n) == n - 1:
            return True  # number is composite
        exp <<= 1  # multiply by 2
    return False  # number is probably prime


def miller_rabbin_test(n, k=20):
    for i in range(k):
        a = rand.randrange(1, n - 1)
        if not decomposing_number(n, a):
            return False  # number is composite
    return True  # number is probably prime

def bin_to_dec(bin_n):
    dec_n = 0
    res = 0
    for i in range(len(bin_n)):
        res = bin_n[len(bin_n) - i - 1] * 2 ** i
        dec_n += res
    return dec_n

def extended_gcd(a, b): 
    if a == 0 : 
        return b,0,1       
    gcd,x1,y1 = extended_gcd(b%a, a) 
     
    x = y1 - (b//a) * x1 
    y = x1 
     
    return gcd,x,y 

In [26]:
def generate_bit_seq(n):
    seq = [0]*n
    for i in range(n):
        seq[i] = rand.randint(0, 1)
    return seq


def L20(n):
    seq = generate_bit_seq(20)
    result = [0]*n
    for i in range(20):
        result[i] = seq[i]
    for i in range(20,n):
        result[i] = result[i-3]^result[i-5]^result[i-9]^result[i-20]
    return result

In [27]:
def generate_prime_number(x):
    res = [1,0,0]
    while(miller_rabbin_test(bin_to_dec(res)) == False):
        res = L20(x)
    return res

In [28]:
def generate_blum_primes(n):
    p = bin_to_dec(generate_prime_number(n))
    while((p-3) % 4 != 0):
        p = bin_to_dec(generate_prime_number(n))
    
    q = bin_to_dec(generate_prime_number(n))
    while((q-3) % 4 != 0):
        q = bin_to_dec(generate_prime_number(n))
        
    return p, q

In [29]:
def fast_square_blum(y, p, q): # x^2 = y(mod n), n = p*q, p & q = 4k + 3
    s_1 = pow(y,(p+1)/4,p)
    s_2 = pow(y,(q+1)/4,q)
    _, u, v = extended_gcd(p, q)
    return u*p*s_1 + v*q*s_2, u*p*s_1 - v*q*s_2, (-1)*u*p*s_1 + v*q*s_2, (-1)*u*p*s_1 - v*q*s_2         # ++, +-, -+, --

In [ ]:
def format(m,n):
    l = math.ceil((len(bin(n))-2)/8)
    if(math.ceil((len(bin(m))-2)/8)<l-10):
        r = 2**64-1
        x = 255*2**(8*(l-8)) + m*2**64 + r
        return x
    
def unformat(m,n):
    l = len(bin(n))-2
    print(l)
    m = bin(m)
    print(m,len(m))
    x=''
    print(l)
    for i in range(10,(len(m)-64)):
        x+=(m[i])
    return int(x,2)

## Прописуємо інтерфейс користувача

In [ ]:
class User:
    
    __key_pair = None
    
    __k = None

    def __init__(self):
        __key_pair = generate_blum_primes
        self.public_n = result[0]
        self.public_e = result[1]
        self.__private_key = result[2]
        
    
    def RSA_decrypt(self, C):
        
    def RSA_sign(self, M):
        
    
    def Send_Key(self, e_1, n_1):
        
    
    def Receive_Key(self, k_1, S_1, e, n):
        